In [47]:
'''
IMPORTS
'''

%matplotlib inline

# Standard imports
import copy
import itertools

# Scientific computing imports
import numpy
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd

In [48]:
class MultiPlex(object):
    def __init__(self):
        self.plex=[]
        self.name=[]
        self.global_node={}
        
    def add_plex(self,name):
        self.plex.append(nx.Graph())
        self.name.append(name)
        #print self.plex
        if len(self.plex)>1:
            for node in self.plex[-2].nodes():
                print node,self.plex[-2].nodes()
                if self.plex[-2].node[node]:
                    self.plex[-1].add_node(node,self.plex[-2].node(node))
                else:
                    self.plex[-1].add_node(node)
                
        
    def add_plex_node(self,node_name,data={}):        
        self.global_node[node_name]=data
        for plex in self.plex:
            if node_name in plex.nodes(): #node exists
                print 'node exists'
                break
            else:
                plex.add_node(node_name,data)
                    
    def edit_plex_node(self,node_name,node_data):
        if node_name not in self.global_node:
            print 'node does not exist'
            return
        else:
            for plex in self.plex:
                plex.node[node_name]=node_data
                
            self.global_node[node_name]=node_data
    
    def add_plex_edge(self,(u,v,plex),data={}):
        #check if u and v exist
        if plex not in self.name:
            print 'plex does not exist'
            return
        elif (u not in self.global_node) or (v not in self.global_node):
            print 'u or v does not exist'
            return
        else:
            plex_index=self.name.index(plex)
            self.plex[plex_index].add_edge(u,v,data)
        
                
    def plexes(self):
            return iter(self.plex)
    
    def names(self):
        return iter(self.name)
    
    def global_nodes(self):
        return iter(self.global_node)
                
 
    def __repr__(self):
        skip_none = True
        repr_string = type(self).__name__ + " ["
        except_list = "model"

        elements = [e for e in dir(self) if str(e) not in except_list]
        for e in elements:
            # Make sure we only display "public" fields; skip anything private (_*), that is a method/function, or that is a module.
            if not e.startswith("_") and eval('type(self.{0}).__name__'.format(e)) not in ['DataFrame', 'function', 'method', 'builtin_function_or_method', 'module', 'instancemethod']:
                    value = eval("self." + e)
                    if value != None and skip_none == True:
                        repr_string += "{0}={1}, ".format(e, value)

        # Clean up trailing space and comma.
        return repr_string.strip(" ").strip(",") + "]"
        

In [67]:
class ShipNET(object):
    def __init__(self,initial_grid_size):
        self.initial_grid_size=initial_grid_size
        self.ship=nx.grid_graph(dim=initial_grid_size)
        self.layout=nx.spectral_layout(self.ship)
        
        #label streets and penetrations
        self.dual_streets={}
        self.geometric_characteristics={'H':'horizontal','L':'longitudinal','T':'transverse'}
        structural_denotation=['T','L','H']
        self.transfer_types={'E':None,'M':None,'I':None,'H':None}
        for i in range(len(self.initial_grid_size)):
            for structure in range(self.initial_grid_size[i]+1):
                s_name='{}{}'.format(structural_denotation[i],structure)
                self.dual_streets[s_name]={}
                self.dual_streets[s_name]['orientation']=self.geometric_characteristics[structural_denotation[i]]
                self.dual_streets[s_name]['availability']=list(self.transfer_types)
                
        for u,v in self.ship.edges():
            coord_change=[(a is b) for a, b in zip(u,v)].index(False)
            if coord_change==0: #move in x
                self.ship[u][v]['penetration']='T{}'.format(max(u[0],v[0]))
                self.ship[u][v]['streets']=['L{}'.format(u[1]),'L{}'.format((u[1]+1)),'H{}'.format(u[2]),'H{}'.format((u[2]+1))]   

            if coord_change==1: #move in y
                self.ship[u][v]['penetration']='L{}'.format(max(u[1],v[1]))
                self.ship[u][v]['streets']=['H{}'.format(u[2]),'H{}'.format((u[2]+1)),'T{}'.format(u[0]),'T{}'.format((u[0]+1))]

            if coord_change==2: #move in z
                self.ship[u][v]['penetration']='H{}'.format(max(u[2],v[2]))
                self.ship[u][v]['streets']=['T{}'.format(u[0]),'T{}'.format((u[0]+1)),'L{}'.format(u[1]),'L{}'.format((u[1]+1))]

            #print (u,v),self.ship[u][v]['streets'],self.ship[u][v]['penetration']

        #generate default information dual
        self.ship_dual=nx.Graph()
        for key in self.dual_streets:
            if self.dual_streets[key]['orientation'] is 'horizontal':
                for cross_street in self.dual_streets:
                    if self.dual_streets[cross_street]['orientation'] is not 'horizontal':
                        self.ship_dual.add_edge(key,cross_street)

            if self.dual_streets[key]['orientation'] is 'transverse':
                for cross_street in self.dual_streets:
                    if self.dual_streets[cross_street]['orientation'] is not 'transverse':
                        self.ship_dual.add_edge(key,cross_street)

            if self.dual_streets[key]['orientation'] is 'longitudinal':
                for cross_street in self.dual_streets:
                    if self.dual_streets[cross_street]['orientation'] is not 'longitudinal':
                        self.ship_dual.add_edge(key,cross_street)
                        
    def affordance_routing(self,affordance_multiplex):
        self.affordance_multiplex=affordance_multiplex
        
        #first get the available subgraph
        for transfer in self.transfer_types:
            s_g=nx.Graph()
            for j,k,d in self.ship.edges(data=True):
                available_streets=[]
                #print (j,k),d
                for street in d['streets']:
                    if transfer in self.dual_streets[street]['availability']:
                        available_streets.append(street)
                if available_streets:
                    s_g.add_edge(j,k,streets=available_streets)
            self.transfer_types[transfer]=s_g
            
        #create dataframe for results
        self.affordance_edges=[]
        for plex,name in zip(self.affordance_multiplex.plexes(),self.affordance_multiplex.names()):
            #print name
            self.affordance_nodes=plex.nodes()
            for u,v in plex.edges():
                #print edge,name
                self.affordance_edges.append((u,v,name))
        
        #print 'edges',self.affordance_edges
                
        self.e_route=pd.DataFrame(index=self.ship.edges(),columns=self.affordance_edges)
        self.e_route=self.e_route.fillna(0)
        #print self.e_route

        self.n_location=pd.DataFrame(index=self.ship.nodes(),columns=self.affordance_nodes)
        self.n_location=self.n_location.fillna(0)
        #print self.n_location

        self.a_incidence=pd.DataFrame(index=self.affordance_nodes,columns=self.affordance_nodes)
        self.a_incidence=self.a_incidence.fillna(0)
        #print self.a_incidence

        self.ae_incidence=pd.DataFrame(index=self.affordance_edges,columns=self.affordance_edges)
        self.ae_incidence=self.ae_incidence.fillna(0)
        #print self.ae_incidence
        
        arrangements=1
        while arrangements<100:
            arrangements+=1
            for n in self.affordance_multiplex.global_nodes():
                data=self.affordance_multiplex.global_node[n]
                #print data
                if data['loc_possible']:
                    n_loc=data['loc_possible'][numpy.random.randint(len(data['loc_possible']))]
                else:
                    n_loc=self.ship.nodes()[numpy.random.randint(len(self.ship.nodes()))]
                    
                data['loc'].append(n_loc)
                #print data
                self.affordance_multiplex.edit_plex_node(n,data)
                self.n_location.ix[n_loc,n]+=1

            #post-process locations
            for n1 in self.affordance_multiplex.global_nodes(): #get affordance incidence
                for n2 in self.affordance_multiplex.global_nodes():
                    if n1==n2:
                        continue
                    if self.affordance_multiplex.global_node[n1]['loc'][-1]==self.affordance_multiplex.global_node[n2]['loc'][-1]:
                        self.a_incidence.ix[n1,n2]+=1
                        #a_incidence.ix[n2,n1]+=1

            
            for plex,name in zip(self.affordance_multiplex.plexes(),self.affordance_multiplex.names()):
                for u,v in plex.edges():
                    base_graph=self.transfer_types[name]
                    #random walk from start to finish
                    walk=nx.Graph()
                    walk.add_node(self.affordance_multiplex.global_node[u]['loc'][-1])
                    last_added=self.affordance_multiplex.global_node[u]['loc'][-1]
                    while self.affordance_multiplex.global_node[v]['loc'][-1] not in walk.nodes():
                        adding_node=base_graph.neighbors(last_added)[numpy.random.randint(len(base_graph.neighbors(last_added)))]
                        walk.add_edge(last_added,adding_node)
                        last_added=adding_node

                    plex[u][v]['paths'].append(walk.edges())
                    for e in walk.edges():
                        if e in self.e_route.index:
                            self.e_route.ix[e,(u,v,name)] += 1
                        else:
                            self.e_route.ix[e[::-1],(u,v,name)] += 1

            #post-process edges
            for plex1,name1 in zip(self.affordance_multiplex.plexes(),self.affordance_multiplex.names()):
                #print name1,plex1.edges()
                for e1_u,e1_v in plex1.edges():
                    #print aff_g[e1_u][e1_v]
                    path1=plex1[e1_u][e1_v]['paths'][-1]
                    for plex2,name2 in zip(self.affordance_multiplex.plexes(),self.affordance_multiplex.names()):
                        #print name2,plex2.edges()
                        for e2_u,e2_v in plex2.edges():
                            #print (e1_u,e1_v,name1),(e2_u,e2_v,name2)
                            if (e1_u,e1_v,name1)==(e2_u,e2_v,name2):
                                continue
                            #print aff_g[e2_u][e2_v]['paths']
                            path2=plex2[e2_u][e2_v]['paths'][-1]
                            for path_edge in path1:
                                if (path_edge in path2) or (path_edge[::-1] in path2):
                                    e_index=(e1_u,e1_v,name1)
                                    e_column=(e2_u,e2_v,name2)
                                    if e_index not in self.ae_incidence.index:
                                        e_index=(e1_v,e1_u,name1)
                                    if e_column not in self.ae_incidence.columns:
                                        e_column=(e2_v,e2_u,name2)

                                    #print e_index,e_column
                                    self.ae_incidence.ix[e_index,e_column]+=1 #increment for each shared edge if no continue
                                    break #only increment once


In [70]:
test_ship=MultiPlex()
test_ship.add_plex('E')
test_ship.add_plex('M')
test_ship.add_plex_node('store_E',data={'loc':[],'loc_possible':[(3,3,0),(4,3,0),(5,3,0)]})
test_ship.add_plex_node('conv_m',data={'loc':[],'loc_possible':[]})
test_ship.add_plex_node('propulsion',data={'loc':[],'loc_possible':[]})
test_ship.add_plex_node('conv_e',data={'loc':[],'loc_possible':[]})
test_ship.add_plex_node('control',data={'loc':[],'loc_possible':[]})
test_ship.add_plex_node('move_int',data={'loc':[],'loc_possible':[]})
test_ship.add_plex_node('move_ext',data={'loc':[],'loc_possible':[]})
test_ship.add_plex_edge(('store_E','conv_m','E'),data={'paths':[]})
test_ship.add_plex_edge(('conv_m','propulsion','E'),data={'paths':[]})
test_ship.add_plex_edge(('store_E','conv_e','E'),data={'paths':[]})
test_ship.add_plex_edge(('conv_e','control','E'),data={'paths':[]})
test_ship.add_plex_edge(('conv_e','move_int','E'),data={'paths':[]})
test_ship.add_plex_edge(('conv_e','move_ext','E'),data={'paths':[]})



for plex,name in zip(test_ship.plexes(),test_ship.names()):
    print name
    print plex.nodes(data=True)
    print plex.edges(data=True)


E
[('conv_m', {'loc': [], 'loc_possible': []}), ('control', {'loc': [], 'loc_possible': []}), ('move_int', {'loc': [], 'loc_possible': []}), ('store_E', {'loc': [], 'loc_possible': [(3, 3, 0), (4, 3, 0), (5, 3, 0)]}), ('propulsion', {'loc': [], 'loc_possible': []}), ('conv_e', {'loc': [], 'loc_possible': []}), ('move_ext', {'loc': [], 'loc_possible': []})]
[('conv_m', 'store_E', {'paths': []}), ('conv_m', 'propulsion', {'paths': []}), ('control', 'conv_e', {'paths': []}), ('move_int', 'conv_e', {'paths': []}), ('store_E', 'conv_e', {'paths': []}), ('conv_e', 'move_ext', {'paths': []})]
M
[('conv_m', {'loc': [], 'loc_possible': []}), ('control', {'loc': [], 'loc_possible': []}), ('move_int', {'loc': [], 'loc_possible': []}), ('store_E', {'loc': [], 'loc_possible': [(3, 3, 0), (4, 3, 0), (5, 3, 0)]}), ('propulsion', {'loc': [], 'loc_possible': []}), ('conv_e', {'loc': [], 'loc_possible': []}), ('move_ext', {'loc': [], 'loc_possible': []})]
[]


In [72]:
ship=ShipNET([10,5,5])
#print ship.dual_streets
ship.affordance_routing(test_ship)
print ship.e_route
print ship.n_location
print ship.a_incidence
print ship.ae_incidence

                        (conv_m, store_E, E)  (conv_m, propulsion, E)  \
((4, 2, 2), (5, 2, 2))                    33                       33   
((4, 2, 2), (4, 2, 1))                    39                       37   
((4, 2, 2), (3, 2, 2))                    36                       28   
((4, 2, 2), (4, 2, 3))                    34                       28   
((4, 2, 2), (4, 1, 2))                    38                       37   
((4, 2, 2), (4, 3, 2))                    30                       34   
((3, 0, 3), (3, 0, 2))                    35                       31   
((3, 0, 3), (4, 0, 3))                    34                       23   
((3, 0, 3), (2, 0, 3))                    27                       31   
((3, 0, 3), (3, 0, 4))                    26                       29   
((3, 0, 3), (3, 1, 3))                    35                       29   
((1, 4, 4), (2, 4, 4))                    28                       27   
((1, 4, 4), (0, 4, 4))                    22       

In [102]:
gs=MultiPlex()
gs.add_plex('E')
gs.add_plex_node('hi')
gs.add_plex_node('world')
gs.add_plex('I')
gs.add_plex_node('!',{'num': 1})
gs.edit_plex_node('hi',{'srs': 'no'})
gs.add_plex_edge(('hi','world','E'))
#gs.__repr__()
for index in range(len(gs.plex)):
    print gs.name[index],gs.plex[index].nodes(data=True)

for plex in gs.plexes():
    print plex.nodes()
    print plex.edges()
    
for n in gs.names():
    print n
    
for n in gs.global_nodes():
    print n

[<networkx.classes.graph.Graph object at 0x00000000140208D0>, <networkx.classes.graph.Graph object at 0x0000000014020EB8>]
world ['world', 'hi']
hi ['world', 'hi']
E [('!', {'num': 1}), ('world', {}), ('hi', {'srs': 'no'})]
I [('!', {'num': 1}), ('world', {}), ('hi', {'srs': 'no'})]
['!', 'world', 'hi']
[('world', 'hi')]
['!', 'world', 'hi']
[]
E
I
!
world
hi


In [20]:
g=nx.Graph()
g.add_node('hi',char='l',num=0)
print g.nodes(data=True)
gt=nx.Graph()
for n,n_d in g.nodes(data=True):
    gt.add_node(n,g.node[n])
    
print gt.nodes(data=True)
gt.node['hi']==g.node['hi']

[('hi', {'char': 'l', 'num': 0})]
[('hi', {'char': 'l', 'num': 0})]


True

In [95]:
if not hi:
    print 't'

NameError: name 'hi' is not defined